Laboratorio #2 - Modulacion por Amplitud

TAREA # 1 
Construir un sistema básico AM de tipo DSC-SC, incluyendo la etapa de modulación y 
demodulación, puede asumir condiciones óptimas, esto es no ruido o pérdidas o modificaciones 
en frecuencia o fase. 

1- Se recomienda crear una función para la modulación y otra para la demodulación. 
2- Utilice sincronización coherente. 
3- Puede asumir un canal óptimo sin pérdidas entre ambos bloques. 
1- Grafique todas las funciones y salidas involucradas, tanto en el tiempo como en la 
frecuencia. Sea ordenado en la forma en que lo presente en la notebook. Describa lo 
que observa en cada una de ellas. 
4- Para el experimento, considere una señal de tono (información) tipo: 
𝑚(𝑡) ó 𝑓(𝑡) = cos (2𝜋𝑓௠𝑡), de forma que NO se incurra en sobre-modulación. 
5- Realice un análisis completo de los resultados obtenidos, en cada parte funcional del 
sistema de comunicación AM DSB-SC.

TAREA # 2 
Usando como base el sistema básico AM de tipo DSC-SC, construido en la sección anterior, 
introduzca una nueva señal de portadora en el bloque demodulador, de manera que pueda 
introducir errores controlados de ∆𝜔 y 𝜃଴.